In [1]:
import aocd
import numpy as np
from itertools import groupby
from typing import Callable

In [2]:
class Traversal:
    grid: np.ndarray
    position: complex
    facing: complex
    wrapping: Callable[[complex, complex], tuple[complex, complex] | None]
    OPEN, WALL, WRAP = '.', '#', ' '

    def __init__(self, grid: np.ndarray, wrapping: Callable):
        self.grid = grid
        self.wrapping = wrapping
        self.position = 0
        self.facing = 1
        while self.tile != self.OPEN:
            self.position += 1

    @property
    def coords(self) -> tuple[int, int]:
        return int(self.position.imag), int(self.position.real)
    
    @property
    def tile(self) -> str:
        return self.grid[self.coords]
    
    @property
    def next_tile(self) -> str:
        next_position = wrapped[0] if (wrapped := self.wrapping(self.position, self.facing)) else self.position + self.facing
        return self.grid[int(next_position.imag), int(next_position.real)]
    
    def turn(self, direction: str):
        self.facing *= 1j if direction == 'R' else -1j
    
    def forward(self, amount: int):
        while amount:
            if self.next_tile == self.WALL:
                break
            if wrapped := self.wrapping(self.position, self.facing):
                self.position, self.facing = wrapped
            else:
                self.position += self.facing
            amount -= 1
    
    def move(self, instruction: str | int):
        if instruction in ('L', 'R'):
            self.turn(instruction)
        else:
            assert isinstance(instruction, int)
            self.forward(instruction)

In [3]:
def parse_instruction(isdigits, group):
    return int(''.join(group)) if isdigits else next(group)

board, instructions = aocd.get_data(day=22, year=2022).split('\n\n')
instructions = [parse_instruction(*i) for i in groupby(instructions, str.isdigit)]
max_length = max(len(line) for line in board.splitlines())
board = [line.ljust(max_length) for line in board.splitlines()]

grid = np.array([[c for c in line] for line in board])
R, L, D, U = 1, -1, 1j, -1j

In [4]:
def wrap2d(position, facing):
    next_position = position + facing
    while grid[int(next_position.imag) % grid.shape[0], int(next_position.real) % grid.shape[1]] == ' ':
        next_position += facing
    return complex(next_position.real % grid.shape[1], next_position.imag % grid.shape[0]), facing

traversal = Traversal(grid, wrapping=wrap2d)
for i in instructions:
    traversal.move(i)

print("Part 1:", 1000*(traversal.coords[0]+1) + 4*(traversal.coords[1]+1) + {R: 0, D: 1, L: 2, U: 3}[traversal.facing])

Part 1: 64256


In [5]:
wrap3d = {}
for n in range(50):
    connections = [
        ((complex(real= 50+n, imag=  0), U), (complex(imag=150+n, real=  0), L)),
        ((complex(real=100+n, imag=  0), U), (complex(real=  0+n, imag=199), D)),
        ((complex(imag=  0+n, real=149), R), (complex(imag=149-n, real= 99), R)),
        ((complex(real=100+n, imag= 49), D), (complex(imag= 50+n, real= 99), R)),
        ((complex(real= 50+n, imag=149), D), (complex(imag=150+n, real= 49), R)),
        ((complex(imag=  0+n, real= 50), L), (complex(imag=149-n, real=  0), L)),
        ((complex(imag= 50+n, real= 50), L), (complex(real=  0+n, imag=100), U))
    ]

    for (p1, f1), (p2, f2) in connections:
        wrap3d[p1, f1] = p2, -f2
        wrap3d[p2, f2] = p1, -f1

In [6]:
traversal = Traversal(grid, wrapping=lambda p, f: wrap3d.get((p, f)))
for i in instructions:
    traversal.move(i)

print("Part 2:", 1000*(traversal.coords[0]+1) + 4*(traversal.coords[1]+1) + {R: 0, D: 1, L: 2, U: 3}[traversal.facing])

Part 2: 109224
